In [56]:
import pandas as pd
import numpy as np
import requests
import json
from pandas.io.json import json_normalize
from PIL import Image
from os import path
import sys
sys.path.append(path.abspath('MSc-CS-Project---ColourPaletteExtractor-master/'))
from colourpaletteextractor.model import model
from bs4 import BeautifulSoup
import urllib.request
from urllib.request import urlopen
import re
import time
from pathlib import Path

In [58]:
# Local file
df = pd.read_json(r'the-tate-collection.json')
df = pd.json_normalize(df['fields'])
# df

In [60]:
# view image
# im = Image.open(requests.get("https://www.tate.org.uk/art/images/work/T/T08/T08333_10.jpg", stream = True).raw)
# im

In [62]:
def url_get_int(url): 
    url = url.split('_')[1]
    url = url.split('.')[0]
    return int(url)

def binary_find_largest(arr):
    temp_list = arr
    length = len(temp_list)
    if len(arr) == 1:
        return temp_list[0]
    elif url_get_int(temp_list[length - 1]) >= url_get_int(temp_list[length - 2]):
        del temp_list[length - 2]
    elif url_get_int(temp_list[length - 1]) <= url_get_int(temp_list[length - 2]):
        del temp_list[length - 1]
    
    return binary_find_largest(temp_list)
    
def get_best_imageURL(number):
    url = df['url'][number]
    accession_number = df['accession_number'][number]
    
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    attrs = []
    page.close()
    
    for node in soup.findAll('img'):
        attrs += list(node.attrs.values())
        
    temp_images = [k for k in attrs if accession_number in k]
    
    # returns NaN if no image avilable
    if len(temp_images) == 0:
        return np.nan
    else:
        return binary_find_largest(temp_images)

In [103]:
new_df = pd.DataFrame(columns = ['img_URL', 'Colour_Palettes'])

In [115]:
def set_df_img_url_range(start, last):
    timer = time.process_time()
    
    for counter in range(start, last + 1):
        try:
            url = get_best_imageURL(counter)
            if url is not np.nan:
                savepath = Path("image/"+ str(df["accession_number"][counter]) + ".jpg")
                if not savepath.is_file():
                    urllib.request.urlretrieve(url, savepath)
                
                if len(new_df) == counter:
                    colors = model.generate_colour_palette_from_image(savepath)
                elif new_df.loc[counter]['Colour_Palettes'] is np.nan:
                    colors = model.generate_colour_palette_from_image(savepath)

                new_df.loc[counter] = [url, colors]

            else:
                new_df.loc[counter] = [url, np.nan]   

        except:
            print(str(counter))
            counter -= 1
            continue
    
    print(time.process_time() - timer)

In [116]:
set_df_img_url_range(0, 100)

0
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
18
19
20
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
85
86
89
91
92
93
94
95
96
97
98
99
100
6.375


In [114]:
new_df

,img_URL,Colour_Palettes
0,https://www.tate.org.uk/art/images/work/P/P80/...,"[[43, 39, 38], [30, 28, 29], [35, 34, 33], [76..."
1,NaN,NaN
2,https://www.tate.org.uk/art/images/work/T/T08/...,"[[81, 61, 49], [136, 114, 94], [194, 177, 155]..."
3,https://www.tate.org.uk/art/images/work/T/T08/...,"[[182, 173, 159], [187, 180, 163], [224, 214, ..."
4,https://www.tate.org.uk/art/images/work/T/T13/...,"[[71, 71, 66], [131, 118, 101], [116, 114, 98]..."
...,...,...
96,https://www.tate.org.uk/art/images/work/D/D18/...,"[[34, 23, 18], [17, 16, 18], [16, 16, 16], [10..."
97,https://www.tate.org.uk/art/images/work/D/D19/...,"[[20, 18, 19], [18, 18, 18], [84, 65, 49], [90..."
98,https://www.tate.org.uk/art/images/work/D/D26/...,"[[23, 24, 26], [24, 25, 24], [192, 177, 149], ..."
99,https://www.tate.org.uk/art/images/work/P/P02/...,"[[35, 35, 31], [70, 69, 64], [119, 119, 113], ..."
